# Explicando Vector DBs

Neste exemplo, vamos criar uma tabela, carregar alguns textos e procurar por similaridade.

Utilizaremos os modelos da OpenAI para gerar embeddings e respostas.

Os dados vetorizados serão armazenados no Astra/Cassandra para busca baseada na similaridade de vetores (Vector Search).

## Configurando ambiente

Inicialmente, vamos instalar e configurar o ambiente para execução dos modelos

In [ ]:
pip install openai cassio --upgrade 

In [8]:
# Carregando variáveis de ambiente
from dotenv import load_dotenv, find_dotenv
import os
load_dotenv(find_dotenv(), override=True)

True

## Conectando ao Astra/Cassandra

Para utilizar o Cassandra como Vector DB, crie um banco de dados com suporte a vvector search em astra.datastax.com, gere um token de conexão e identifique o seu DB ID. Com isso, execute o comando abaixo para criar a conexão

In [2]:
import cassio

cassio.init(token=os.environ["ASTRA_DB_APPLICATION_TOKEN"], database_id=os.environ["ASTRA_DB_ID"])

Pronto!

A conexão com o Astra é muito simples e você já pode armazenar seus documentos.

## Criando a tabela/índice no Astra

A estrutura no banco de dados que vai armazenar os dados, no caso do Astra/Cassandra, são as tabelas.

O Astra possui uma modelagem baseada em partições que permite escalar os dados até terabytes e milhões de registros.

Além disso, os índices possibilitam queries com suporte a vector search, como veremos a seguir

In [4]:
# Criando a tabela
from cassio.table.tables import MetadataVectorCassandraTable

vector_table = MetadataVectorCassandraTable(
    keyspace="demo",
    table="product_vector",
    vector_dimension=1536,
)

## Embeddings

Vamos converter nossa base de documentos em embeddings. 

Se você não sabe o que são embeddings, assista aqui: https://www.youtube.com/watch?v=fvRyziDmvoA

Vamos gerar embeddings para 2 produtos

In [12]:
# Inicializando a OpenAI
import openai
openai.api_key = os.environ["OPENAI_API_KEY"]

def generate_embedding(text):
    model = "text-embedding-ada-002"
    response = openai.Embedding.create(model=model, input=text)
    return response.data[0]['embedding']


In [15]:
rs = vector_table.get(row_id=product_id)
rs["body_blob"]

'iphone 15, 256GB, Azul'

## Inserindo produtos

In [16]:
product_description = "iphone 15, 256GB, Azul"
product_id = "prod1"
vector_table.put_async(
    body_blob=product_description,
    vector=generate_embedding(product_description),
    row_id=product_id,
    metadata={"id": product_id}
)

#Verificando se o registro foi inserido
rs = vector_table.get(row_id=product_id)
rs["body_blob"]

'iphone 15, 256GB, Azul'

In [17]:
product_description = "Cadeira Gamer Giratória Royale Com Regulagem de Altura Preto/Cinza"
product_id = "prod2"
vector_table.put_async(
    body_blob=product_description,
    vector=generate_embedding(product_description),
    row_id=product_id,
    metadata={"id": product_id}
)

#Verificando se o registro foi inserido
rs = vector_table.get(row_id=product_id)
rs["body_blob"]

'Cadeira Gamer Giratória Royale Com Regulagem de Altura Preto/Cinza'

## Agora que já inserimos 2 produtos, vamos buscar por similaridade

In [21]:
q = "Smartphone cinza"
results = vector_table.ann_search(n=1, vector=generate_embedding(q))
for r in results:
    print(r['body_blob'])

iphone 15, 256GB, Azul
